<a href="https://colab.research.google.com/github/jtao/VIST271/blob/main/Image_Filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Filtering with PIL

Jian Tao, Texas A&M University

Feb 28, 2022

We will explore image filtering with Python PIL library.

First a few librabries will be imported.

In [ ]:
oak_path = "https://github.com/jtao/VIST271/raw/main/images/post_oak.jpg"
oak_file = "post_oak.jpg"

## Read Image

We first download an image from a URL and then open it with the Image.open function in PIL. Let's convert it to the grayscale and work on one channel only.

In [ ]:
import numpy as np
import os
import random
from matplotlib import pyplot as plt
from PIL import Image, ImageFilter
import urllib.request
urllib.request.urlretrieve(oak_path, oak_file)
oak = Image.open(oak_file).convert('L')
oak

In [ ]:
oak_mat = np.array(oak)

In [ ]:
oak_mat.shape

Let's pull out row 200-209 and column 200-209 of the matrix and print the numbers out.

In [ ]:
print (oak_mat[200:210, 200:210])

Let's try to plot directly from the array oak_mat

In [ ]:
Image.fromarray(oak_mat[200:210, 200:210])

We can find out where it is by coloring it white in the original matrix.

In [ ]:
#oak_mat[200:210, 200:210] = 255
#Image.fromarray(oak_mat)

## Segmentation filter
### Let's try to set the value of a pixel to 255 when it is greater than 100 and 0 otherwise.

In [ ]:
def segmentation(im, cutoff=100):
  im_mat = np.array(im)
  for i in range(im_mat.shape[0]):
    for j in range(im_mat.shape[1]):
      if im_mat[i, j] > cutoff:
        im_mat[i, j] = 255
      else:
        im_mat[i, j] = 0
  return im_mat

In [ ]:
Image.fromarray(segmentation(oak, cutoff=70))

## Let's add some noise to the original image
The code below to add noise could be considered as a segmentation filter as well.

In [ ]:
def add_noise(im, ratio=0.4):
  im_mat = np.array(im)
  mask = np.random.random(im_mat.shape)
  for i in range(mask.shape[0]):
    for j in range(mask.shape[1]):
      if mask[i, j] > 1-ratio:
# add a random number to generate salt and pepper noise        
        im_mat[i, j] = 255*random.getrandbits(1)
  return im_mat

In [ ]:
oak_noisy = add_noise(oak, ratio=0.01)
Image.fromarray(oak_noisy)

## Identity Filter/Kernel

In [ ]:
def identity_filter(size=3):
  if size % 2 == 0:
    print ("the size must be an odd number!")
    return 
  filter_mat = np.zeros((size, size))
  filter_mat[size//2, size//2] = 1
  return filter_mat

In [ ]:
identity_filter(5)

## Mean Filter/Kernel

In [ ]:
def mean_filter(size=3):
  filter_mat = np.full((size, size), 1/(size*size))
  return filter_mat

In [ ]:
mean_filter(3)

## Some other commonly used filters

In [ ]:
sharpen_filter = np.array([
    [0, -1, 0],
    [-1, 5, -1],
    [0, -1, 0]
])

blur_filter = np.array([
    [0.0625, 0.125, 0.0625],
    [0.125,  0.25,  0.125],
    [0.0625, 0.125, 0.0625]
])

outline_filter = np.array([
    [-1, -1, -1],
    [-1,  8, -1],
    [-1, -1, -1]
])

In [ ]:
myfilter = 2*identity_filter(3) - mean_filter(3)

In [ ]:
myfilter

## Apply the convolution kernel 



In [ ]:
def apply_filter(im, kernel):
  im = np.asarray(im)
  m, n = im.shape
  o, p = kernel.shape
  k_half = o // 2
  print (m,n,o,p)
  im = np.pad(im, (k_half, k_half), 'constant', constant_values=0)
  im_new = np.zeros((m, n))

  for i in range(1, m-1):
    for j in range(1, n-1):
      mat = im[i-k_half:i+k_half, j-k_half:j+k_half]
      temp = 0
      for k in range(o):
        for l in range(p):
          temp += im[i+k, j+l] * kernel[k, l]         
          im_new[i, j] = min(255, max(0, temp))          
    # temp = im[i-1, j-1]*kernel[0, 0] \
    #       +im[i-1, j  ]*kernel[0, 1] \
    #       +im[i-1, j+1]*kernel[0, 2] \
    #       +im[i,   j-1]*kernel[1, 0] \
    #       +im[i,   j  ]*kernel[1, 1] \
    #       +im[i,   j+1]*kernel[1, 2] \
    #       +im[i+1, j-1]*kernel[2, 0] \
    #       +im[i+1, j  ]*kernel[2, 1] \
    #       +im[i+1, j+1]*kernel[2, 2]              
  return im_new.astype(np.uint8)
  

In [ ]:
oak_blur = apply_filter(oak, blur_filter)
Image.fromarray(oak_blur)

In [ ]:
oak_sharpen = apply_filter(oak, sharpen_filter)
Image.fromarray(oak_sharpen)

In [ ]:
def seg_filter(im, size = 3, filter="median"):
  tfilter = {"median":np.median, "max":np.max, "min":np.min, "mean":np.mean}
  im = np.asarray(im)
  m, n = im.shape
  k_half = size // 2
  print (m,n,size)
  im = np.pad(im, (k_half, k_half), 'constant', constant_values=0)
  im_new = np.zeros((m, n))

  for i in range(m):
    for j in range(n):
      mat = im[i:i+size, j:j+size]
      im_new[i, j] = tfilter[filter](mat)
  return im_new.astype(np.uint8)

In [ ]:
Image.fromarray(oak_noisy)

In [ ]:
Image.fromarray(seg_filter(oak_noisy, size = 3, filter = "median"))

In [ ]:
Image.fromarray(seg_filter(oak, size = 3, filter = "max"))

In [ ]:
Image.fromarray(seg_filter(oak, size = 3, filter = "min"))

In [ ]:
Image.fromarray(seg_filter(oak_blur, size = 3, filter = "median"))

## We can compare the convolution mean filter and the segmentation mean filter. They are essentailly the same.

In [ ]:
Image.fromarray(apply_filter(oak_blur, mean_filter(3)))

In [ ]:
Image.fromarray(seg_filter(oak_blur, size = 3, filter = "mean"))